In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

import matplotlib.pyplot as plt

import operator
from functools import reduce
from functools import partial

from timeit import default_timer
import utilities3
import importlib
importlib.reload(utilities3)

torch.manual_seed(0)
np.random.seed(0)